# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [2]:
# load in the data
azdias = pd.read_csv('Udacity_AZDIAS_052018.csv', sep=',')
customers = pd.read_csv('Udacity_CUSTOMERS_052018.csv', sep=',')

d:\ProgramData\Miniconda3\envs\hackthon\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Be sure to add in a lot more cells (both markdown and code) to document your
# approach and findings!

In [4]:
azdias.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VHN,VK_DHT4A,VK_DISTANZ,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,910215,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,1,2
1,910220,-1,9.0,0.0,NaN,NaN,NaN,NaN,21.0,11.0,...,4.0,8.0,11.0,10.0,3.0,9.0,4.0,5,2,1
2,910225,-1,9.0,17.0,NaN,NaN,NaN,NaN,17.0,10.0,...,2.0,9.0,9.0,6.0,3.0,9.0,2.0,5,2,3
3,910226,2,1.0,13.0,NaN,NaN,NaN,NaN,13.0,1.0,...,0.0,7.0,10.0,11.0,NaN,9.0,7.0,3,2,4
4,910241,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,3.0,...,2.0,3.0,5.0,4.0,2.0,9.0,3.0,4,1,3


In [5]:
customers.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,PRODUCT_GROUP,CUSTOMER_GROUP,ONLINE_PURCHASE,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,9626,2,1.0,10.0,NaN,NaN,NaN,NaN,10.0,1.0,...,2.0,6.0,9.0,7.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,1,4
1,9628,-1,9.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,0.0,9.0,NaN,3,FOOD,SINGLE_BUYER,0,1,4
2,143872,-1,1.0,6.0,NaN,NaN,NaN,NaN,0.0,1.0,...,11.0,6.0,9.0,2.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,2,4
3,143873,1,1.0,8.0,NaN,NaN,NaN,NaN,8.0,0.0,...,2.0,NaN,9.0,7.0,1,COSMETIC,MULTI_BUYER,0,1,4
4,143874,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,7.0,...,4.0,2.0,9.0,3.0,1,FOOD,MULTI_BUYER,0,1,3


In [6]:
azdias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891221 entries, 0 to 891220
Columns: 366 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: float64(267), int64(93), object(6)
memory usage: 2.4+ GB


In [7]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652 entries, 0 to 191651
Columns: 369 entries, LNR to ALTERSKATEGORIE_GROB
dtypes: float64(267), int64(94), object(8)
memory usage: 539.5+ MB


In [15]:
customers.shape

(191652, 369)

In [8]:
customers['CUSTOMER_GROUP'].unique()

array(['MULTI_BUYER', 'SINGLE_BUYER'], dtype=object)

In [9]:
customers['PRODUCT_GROUP'].unique()

array(['COSMETIC_AND_FOOD', 'FOOD', 'COSMETIC'], dtype=object)

In [10]:
customers['ONLINE_PURCHASE'].unique()

array([0, 1], dtype=int64)

## Data Cleaning

1. eye vision first using Excel. 
There are only 314 artributes in "DIAS Information Levels - Attributes 2017". But there are 369 columns in "Udacity_CUSTOMERS_052018". I sort out a map file 

In [35]:
# Perform an assessment of how much missing data there is in each column of the
# dataset.

pd.options.display.max_rows = None
cust_missing_rate=(customers.shape[0] - customers.count())/customers.shape[0] * 100

In [42]:
cust_missing_rate

LNR                             0.000000
AGER_TYP                        0.000000
AKT_DAT_KL                     24.312817
ALTER_HH                       24.312817
ALTER_KIND1                    93.860748
ALTER_KIND2                    97.338927
ALTER_KIND3                    99.334732
ALTER_KIND4                    99.876860
ALTERSKATEGORIE_FEIN           27.050070
ANZ_HAUSHALTE_AKTIV            26.050863
ANZ_HH_TITEL                   27.189907
ANZ_KINDER                     24.312817
ANZ_PERSONEN                   24.312817
ANZ_STATISTISCHE_HAUSHALTE     26.050863
ANZ_TITEL                      24.312817
ARBEIT                         26.337320
BALLRAUM                       26.067560
CAMEO_DEU_2015                 26.312274
CAMEO_DEUG_2015                26.312274
CAMEO_INTL_2015                26.312274
CJT_GESAMTTYP                   1.676476
CJT_KATALOGNUTZER               1.676476
CJT_TYP_1                       1.676476
CJT_TYP_2                       1.676476
CJT_TYP_3       

In [46]:
drop_cols = []

for key in cust_missing_rate.keys():
    if cust_missing_rate[key]>30:
        drop_cols.append(key)
print(drop_cols)

['ALTER_KIND1', 'ALTER_KIND2', 'ALTER_KIND3', 'ALTER_KIND4', 'EXTSEL992', 'KK_KUNDENTYP']


In [47]:
customers.drop(columns=drop_cols,inplace=True)

In [48]:
customers.shape

(191652, 363)

1. seperate the columns into 10 big category according to *Information level*.
```
Person
Household
Building
Microcell (RR4_ID)
Microcell (RR3_ID)
125m x 125m Grid
Postcode 
RR1_ID
PLZ8
Community
```

2. roughly compute the rate of missing data per column, if the rate is bigger than 30%, romove the column

---------------------------------------

- field name: SOHO_FLAG in 'DIAS Information Levels - Attributes 2017.xlsx' 
            SOHO_KZ in 'Udacity_CUSTOMERS_052018.csv'
            
    solution: changed to SOHO_KZ


In [49]:
cust_person = customers[['LP_LEBENSPHASE_FEIN',
                        'LP_LEBENSPHASE_GROB',
                        'LP_FAMILIE_FEIN',
                       'LP_FAMILIE_GROB', 
                        'LP_STATUS_FEIN',
                       'LP_STATUS_GROB',
                        'NATIONALITAET_KZ',
                        'PRAEGENDE_JUGENDJAHRE',
                        'RETOURTYP_BK_S',
                        'SEMIO_SOZ',
                        'SEMIO_FAM',
                        'SEMIO_REL',
                        'SEMIO_MAT',
                        'SEMIO_VERT',
                        'SEMIO_LUST',
                        'SEMIO_ERL',
                        'SEMIO_KULT',
                        'SEMIO_RAT',
                        'SEMIO_KRIT',
                        'SEMIO_DOM',
                        'SEMIO_KAEM',
                        'SEMIO_PFLICHT',
                        'SEMIO_TRADV',
                        'SHOPPER_TYP',
                        'SOHO_KZ',
                        'TITEL_KZ',
                        'VERS_TYP',
                        'ZABEOTYP',]]

In [26]:
cust_person.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652 entries, 0 to 191651
Data columns (total 28 columns):
LP_LEBENSPHASE_FEIN      188439 non-null float64
LP_LEBENSPHASE_GROB      188439 non-null float64
LP_FAMILIE_FEIN          188439 non-null float64
LP_FAMILIE_GROB          188439 non-null float64
LP_STATUS_FEIN           188439 non-null float64
LP_STATUS_GROB           188439 non-null float64
NATIONALITAET_KZ         191652 non-null int64
PRAEGENDE_JUGENDJAHRE    191652 non-null int64
RETOURTYP_BK_S           188439 non-null float64
SEMIO_SOZ                191652 non-null int64
SEMIO_FAM                191652 non-null int64
SEMIO_REL                191652 non-null int64
SEMIO_MAT                191652 non-null int64
SEMIO_VERT               191652 non-null int64
SEMIO_LUST               191652 non-null int64
SEMIO_ERL                191652 non-null int64
SEMIO_KULT               191652 non-null int64
SEMIO_RAT                191652 non-null int64
SEMIO_KRIT               1916

In [27]:
cust_person.to_csv('cust_person.cvs', index=False)

- field name: D19_KK_KUNDENTYP in 'DIAS Information Levels - Attributes 2017.xlsx' 
            D19_KONSUMTYP_MAX in 'Udacity_CUSTOMERS_052018.csv'
            
    solution: changed to D19_KONSUMTYP_MAX
         
 
- field name:  HAUSHALTSSTRUKTUR not in 'Udacity_CUSTOMERS_052018.csv'

    solution: remove

In [12]:
cust_household = customers[['ALTER_HH',
'ANZ_PERSONEN',
'ANZ_TITEL',
# 'HAUSHALTSSTRUKTUR',
'HH_EINKOMMEN_SCORE',
'D19_KONSUMTYP_MAX',
'D19_KONSUMTYP',
'D19_GESAMT_ANZ_12',
'D19_GESAMT_ANZ_24',
'D19_BANKEN_ANZ_12',
'D19_BANKEN_ANZ_24',
'D19_TELKO_ANZ_12',
'D19_TELKO_ANZ_24',
'D19_VERSI_ANZ_12',
'D19_VERSI_ANZ_24',
'D19_VERSAND_ANZ_12',
'D19_VERSAND_ANZ_24',
'D19_GESAMT_OFFLINE_DATUM',
'D19_GESAMT_ONLINE_DATUM',
'D19_GESAMT_DATUM',
'D19_BANKEN_OFFLINE_DATUM',
'D19_BANKEN_ONLINE_DATUM',
'D19_BANKEN_DATUM',
'D19_TELKO_OFFLINE_DATUM',
'D19_TELKO_ONLINE_DATUM',
'D19_TELKO_DATUM',
'D19_VERSAND_OFFLINE_DATUM',
'D19_VERSAND_ONLINE_DATUM',
'D19_VERSAND_DATUM',
'D19_VERSI_OFFLINE_DATUM',
'D19_VERSI_ONLINE_DATUM',
'D19_VERSI_DATUM',
'D19_GESAMT_ONLINE_QUOTE_12',
'D19_BANKEN_ONLINE_QUOTE_12',
'D19_VERSAND_ONLINE_QUOTE_12',
'W_KEIT_KIND_HH',
'WOHNDAUER_2008']]

In [13]:
cust_household.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652 entries, 0 to 191651
Data columns (total 36 columns):
ALTER_HH                       145056 non-null float64
ANZ_PERSONEN                   145056 non-null float64
ANZ_TITEL                      145056 non-null float64
HH_EINKOMMEN_SCORE             188684 non-null float64
D19_KONSUMTYP_MAX              191652 non-null int64
D19_KONSUMTYP                  143955 non-null float64
D19_GESAMT_ANZ_12              191652 non-null int64
D19_GESAMT_ANZ_24              191652 non-null int64
D19_BANKEN_ANZ_12              191652 non-null int64
D19_BANKEN_ANZ_24              191652 non-null int64
D19_TELKO_ANZ_12               191652 non-null int64
D19_TELKO_ANZ_24               191652 non-null int64
D19_VERSI_ANZ_12               191652 non-null int64
D19_VERSI_ANZ_24               191652 non-null int64
D19_VERSAND_ANZ_12             191652 non-null int64
D19_VERSAND_ANZ_24             191652 non-null int64
D19_GESAMT_OFFLINE_DATUM       

------------------------------------------
- field name: 

*GEOSCORE_KLS7* not in 'Udacity_CUSTOMERS_052018.csv'
            
    solution: remove 
         
 



In [60]:
cust_building = customers[['ANZ_HAUSHALTE_AKTIV',
'ANZ_HH_TITEL',
'GEBAEUDETYP',
# 'GEOSCORE_KLS7',
'KBA05_HERSTTEMP',
'KBA05_MODTEMP',
'KONSUMNAEHE',
'MIN_GEBAEUDEJAHR',
'OST_WEST_KZ',
'WOHNLAGE']]

In [61]:
cust_building.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652 entries, 0 to 191651
Data columns (total 9 columns):
ANZ_HAUSHALTE_AKTIV    141725 non-null float64
ANZ_HH_TITEL           139542 non-null float64
GEBAEUDETYP            141725 non-null float64
KBA05_HERSTTEMP        141725 non-null float64
KBA05_MODTEMP          141725 non-null float64
KONSUMNAEHE            145001 non-null float64
MIN_GEBAEUDEJAHR       141725 non-null float64
OST_WEST_KZ            141725 non-null object
WOHNLAGE               141725 non-null float64
dtypes: float64(8), object(1)
memory usage: 13.2+ MB


-----------------------------------------
- field name: 

*CAMEO_DEUINTL_2015* in DIAS Information Levels - Attributes 2017.xlsx 

*CAMEO_INTL_2015* in Udacity_CUSTOMERS_052018.csv
            
    solution: changed to CAMEO_INTL_2015

In [32]:
cust_rr4 = customers[['CAMEO_DEUG_2015',
'CAMEO_DEU_2015',
'CAMEO_INTL_2015']]

In [33]:
cust_rr4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652 entries, 0 to 191651
Data columns (total 3 columns):
CAMEO_DEUG_2015    141224 non-null object
CAMEO_DEU_2015     141224 non-null object
CAMEO_INTL_2015    141224 non-null object
dtypes: object(3)
memory usage: 4.4+ MB


-----------------------------------------
- field name: 

*WACHSTUMSGEBIET_NB* not in Udacity_CUSTOMERS_052018.csv
            
    solution: remove  


In [36]:
cust_rr3 = customers[['KBA05_ALTER1',
'KBA05_ALTER2',
'KBA05_ALTER3',
'KBA05_ALTER4',
'KBA05_ANHANG',
'KBA05_ANTG1',
'KBA05_ANTG2',
'KBA05_ANTG3',
'KBA05_ANTG4',
'KBA05_AUTOQUOT',
'KBA05_BAUMAX',
'KBA05_CCM1',
'KBA05_CCM2',
'KBA05_CCM3',
'KBA05_CCM4',
'KBA05_DIESEL',
'KBA05_FRAU',
'KBA05_GBZ',
'KBA05_HERST1',
'KBA05_HERST2',
'KBA05_HERST3',
'KBA05_HERST4',
'KBA05_HERST5',
'KBA05_KRSAQUOT',
'KBA05_KRSHERST1',
'KBA05_KRSHERST2',
'KBA05_KRSHERST3',
'KBA05_KRSKLEIN',
'KBA05_KRSOBER',
'KBA05_KRSVAN',
'KBA05_KRSZUL',
'KBA05_KW1',
'KBA05_KW2',
'KBA05_KW3',
'KBA05_MAXAH',
'KBA05_MAXBJ',
'KBA05_MAXHERST',
'KBA05_MAXSEG',
'KBA05_MAXVORB',
'KBA05_MOD1',
'KBA05_MOD2',
'KBA05_MOD3',
'KBA05_MOD4',
'KBA05_MOD8',
'KBA05_MOTOR',
'KBA05_MOTRAD',
'KBA05_SEG1',
'KBA05_SEG2',
'KBA05_SEG3',
'KBA05_SEG4',
'KBA05_SEG5',
'KBA05_SEG6',
'KBA05_SEG7',
'KBA05_SEG8',
'KBA05_SEG9',
'KBA05_SEG10',
'KBA05_VORB0',
'KBA05_VORB1',
'KBA05_VORB2',
'KBA05_ZUL1',
'KBA05_ZUL2',
'KBA05_ZUL3',
'KBA05_ZUL4']]

In [37]:
cust_rr3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652 entries, 0 to 191651
Data columns (total 63 columns):
KBA05_ALTER1       135672 non-null float64
KBA05_ALTER2       135672 non-null float64
KBA05_ALTER3       135672 non-null float64
KBA05_ALTER4       135672 non-null float64
KBA05_ANHANG       135672 non-null float64
KBA05_ANTG1        135672 non-null float64
KBA05_ANTG2        135672 non-null float64
KBA05_ANTG3        135672 non-null float64
KBA05_ANTG4        135672 non-null float64
KBA05_AUTOQUOT     135672 non-null float64
KBA05_BAUMAX       135672 non-null float64
KBA05_CCM1         135672 non-null float64
KBA05_CCM2         135672 non-null float64
KBA05_CCM3         135672 non-null float64
KBA05_CCM4         135672 non-null float64
KBA05_DIESEL       135672 non-null float64
KBA05_FRAU         135672 non-null float64
KBA05_GBZ          135672 non-null float64
KBA05_HERST1       135672 non-null float64
KBA05_HERST2       135672 non-null float64
KBA05_HERST3       135672 non

PLZ not in Udacity_CUSTOMERS_052018.csv

remove

In [65]:
cust_postcode = customers[['BALLRAUM',
'EWDICHTE',
'INNENSTADT',
# 'PLZ'
]]

In [66]:
cust_postcode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652 entries, 0 to 191651
Data columns (total 3 columns):
BALLRAUM      141693 non-null float64
EWDICHTE      141693 non-null float64
INNENSTADT    141693 non-null float64
dtypes: float64(3)
memory usage: 4.4 MB


In [51]:
cust_rr1 = customers[['GEBAEUDETYP_RASTER',
'KKK',
'MOBI_REGIO',
'ONLINE_AFFINITAET',
'REGIOTYP']]

In [52]:
cust_rr1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652 entries, 0 to 191651
Data columns (total 5 columns):
GEBAEUDETYP_RASTER    141725 non-null float64
KKK                   137392 non-null float64
MOBI_REGIO            135672 non-null float64
ONLINE_AFFINITAET     188439 non-null float64
REGIOTYP              137392 non-null float64
dtypes: float64(5)
memory usage: 7.3 MB



'PLZ8' not in Udacity_CUSTOMERS_052018.csv
'KBA13_CCM_1400_2500' change to KBA13_CCM_1401_2500



In [57]:
cust_plz8 = customers[['KBA13_ALTERHALTER_30',
'KBA13_ALTERHALTER_45',
'KBA13_ALTERHALTER_60',
'KBA13_ALTERHALTER_61',
'KBA13_ANZAHL_PKW',
'KBA13_AUDI',
'KBA13_AUTOQUOTE',
'KBA13_BJ_1999',
'KBA13_BJ_2000',
'KBA13_BJ_2004',
'KBA13_BJ_2006',
'KBA13_BJ_2008',
'KBA13_BJ_2009',
'KBA13_BMW',
'KBA13_CCM_1000',
'KBA13_CCM_1200',
'KBA13_CCM_1400',
'KBA13_CCM_0_1400',
'KBA13_CCM_1500',
'KBA13_CCM_1401_2500',
'KBA13_CCM_1600',
'KBA13_CCM_1800',
'KBA13_CCM_2000',
'KBA13_CCM_2500',
'KBA13_CCM_2501',
'KBA13_FAB_ASIEN',
'KBA13_FAB_SONSTIGE',
'KBA13_FIAT',
'KBA13_FORD',
'KBA13_HALTER_20',
'KBA13_HALTER_25',
'KBA13_HALTER_30',
'KBA13_HALTER_35',
'KBA13_HALTER_40',
'KBA13_HALTER_45',
'KBA13_HALTER_50',
'KBA13_HALTER_55',
'KBA13_HALTER_60',
'KBA13_HALTER_65',
'KBA13_HALTER_66',
'KBA13_HERST_ASIEN',
'KBA13_HERST_AUDI_VW',
'KBA13_HERST_BMW_BENZ',
'KBA13_HERST_EUROPA',
'KBA13_HERST_FORD_OPEL',
'KBA13_HERST_SONST',
'KBA13_KMH_110',
'KBA13_KMH_140',
'KBA13_KMH_180',
'KBA13_KMH_0_140',
'KBA13_KMH_140_210',
'KBA13_KMH_211',
'KBA13_KMH_250',
'KBA13_KMH_251',
'KBA13_KRSAQUOT',
'KBA13_KRSHERST_AUDI_VW',
'KBA13_KRSHERST_BMW_BENZ',
'KBA13_KRSHERST_FORD_OPEL',
'KBA13_KRSSEG_KLEIN',
'KBA13_KRSSEG_OBER',
'KBA13_KRSSEG_VAN',
'KBA13_KRSZUL_NEU',
'KBA13_KW_30',
'KBA13_KW_40',
'KBA13_KW_50',
'KBA13_KW_60',
'KBA13_KW_0_60',
'KBA13_KW_70',
'KBA13_KW_61_120',
'KBA13_KW_80',
'KBA13_KW_90',
'KBA13_KW_110',
'KBA13_KW_120',
'KBA13_KW_121',
'KBA13_MAZDA',
'KBA13_MERCEDES',
'KBA13_MOTOR',
'KBA13_NISSAN',
'KBA13_OPEL',
'KBA13_PEUGEOT',
'KBA13_RENAULT',
'KBA13_SEG_GELAENDEWAGEN',
'KBA13_SEG_GROSSRAUMVANS',
'KBA13_SEG_KLEINST',
'KBA13_SEG_KLEINWAGEN',
'KBA13_SEG_KOMPAKTKLASSE',
'KBA13_SEG_MINIVANS',
'KBA13_SEG_MINIWAGEN',
'KBA13_SEG_MITTELKLASSE',
'KBA13_SEG_OBEREMITTELKLASSE',
'KBA13_SEG_OBERKLASSE',
'KBA13_SEG_SONSTIGE',
'KBA13_SEG_SPORTWAGEN',
'KBA13_SEG_UTILITIES',
'KBA13_SEG_VAN',
'KBA13_SEG_WOHNMOBILE',
'KBA13_SITZE_4',
'KBA13_SITZE_5',
'KBA13_SITZE_6',
'KBA13_TOYOTA',
'KBA13_VORB_0',
'KBA13_VORB_1',
'KBA13_VORB_1_2',
'KBA13_VORB_2',
'KBA13_VORB_3',
'KBA13_VW',
# 'PLZ8',
'PLZ8_ANTG1',
'PLZ8_ANTG2',
'PLZ8_ANTG3',
'PLZ8_ANTG4',
'PLZ8_BAUMAX',
'PLZ8_HHZ',
'PLZ8_GBZ']]

In [58]:
cust_plz8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652 entries, 0 to 191651
Columns: 113 entries, KBA13_ALTERHALTER_30 to PLZ8_GBZ
dtypes: float64(113)
memory usage: 165.2 MB


'GKZ', 'EINWOHNER' not in Udacity_CUSTOMERS_052018.csv
remove

In [67]:
cust_community = customers[['ARBEIT',
# 'EINWOHNER',
# 'GKZ',
'ORTSGR_KLS9',
'RELAT_AB']]

In [68]:
cust_community.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191652 entries, 0 to 191651
Data columns (total 3 columns):
ARBEIT         141176 non-null float64
ORTSGR_KLS9    141176 non-null float64
RELAT_AB       141176 non-null float64
dtypes: float64(3)
memory usage: 4.4 MB


## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')